<a href="https://colab.research.google.com/github/umakantiiit/RAG-PIPELINE-USING-LANGCHAIN-GEMINI-CHROMADB-AND-HUGGINGFACE/blob/main/Research_Paper_RAG_with_Langchain_%2CGemini%2CChromadb_and_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Notebook

## Libraries and Utilities installation

In [4]:
!pip install --upgrade -q langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.4
    Uninstalling google-ai-generativelanguage-0.6.4:
      Successfully uninstalled google-ai-generativelanguage-0.6.4
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.5.4
    Uninstalling google-generativeai-0.5.4:
      Successfully uninstalled google-generativeai-0.5.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 

### General Libraries

In [1]:
from google.colab import userdata
import os
from IPython.display import Markdown

### API Keys Configuration

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('SMITH_APIKEY')
GOOGLE_API_KEY = userdata.get('GoogleAIStudio')

In [2]:
!mkdir /content/MyData
!mkdir /content/VectorDB

### Required Libraries

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

In [6]:
# @title Data Source Folder
source_data_folder = "/content/MyData" # @param {type:"string"}


## Data Preparation

In [7]:
# Read PDFs from the configured folder
loader = PyPDFDirectoryLoader(source_data_folder)
data_on_pdf = loader.load()
# Size of the data / documents loaded
len(data_on_pdf)

6

In [8]:
# Partitioning the data. With a limited size (chunks) and 200 characters of overlapping to preserve the context
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(data_on_pdf)
# Number of Chunks generated
len(splits)

37

In [9]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# @title Database folder path
path_db = "/content/VectorDB" # @param {type:"string"}


In [11]:
# Store the chunks in the Data Base
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model, persist_directory=path_db)

## Required Libraries

In [12]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## LLM configuration

We will use Gemini

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key='#put your gemini api key here')

## Prompt and Retriever



In [14]:
retriever = vectorstore.as_retriever()

# https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

In [15]:
def format_docs(docs):
    """Format the documents for the prompt."""
    return "\n\n".join(doc.page_content for doc in docs)




rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



# RAG Execution

In [16]:
# @title Questions to the document
question = "Summarize the document in 3 lines?" # @param {type:"string"}
response = rag_chain.invoke(question )
Markdown(response)


The document proposes a brain tumor segmentation model using Graph Attention Network (GAT). The model represents MRI scans as a region adjacency graph (RAG) and uses GAT to identify tumor types. Experiments show the model outperforms existing solutions in accuracy and achieves faster prediction time. 


In [17]:
# @title Questions to the document
question = "What are the performances acheived with this architecture? " # @param {type:"string"}
response = rag_chain.invoke(question )
Markdown(response)


The GAT model achieves an average Dice score of 0.85 and an average HD95 of 7.17. 
This represents a 6% improvement in Dice score and a 51% improvement in HD95 compared to the baseline GNN model. 
Overall, the GAT model outperforms existing solutions with respect to HD95. 


In [18]:
# @title Questions to the document
question = "How many types of brain tumor are described in this document" # @param {type:"string"}
response = rag_chain.invoke(question )
Markdown(response)


The document describes three types of brain tumors: core tumor, enhancing tumor, and whole tumor. These tumor types are identified through a graph attention network using MRI scans. This information is found in the "Phase 1: Input" section of the document. 
